In [ ]:
# @title
import os,subprocess,sys,fileinput,time,requests,json,subprocess,threading,time,socket,urllib.request,ipywidgets as widgets
from datetime import datetime
from subprocess import getoutput
from IPython.utils import capture
from PIL import Image
from IPython.display import clear_output,display, HTML, Image
from io import BytesIO

check_value = {
    'DriveSyn' : True,
    'Lib' : True,
    'root_folder' :'/content',
    'User_folder' : '/content/drive/MyDrive/SD-Data',
    'SDVNFolder': '/content/SDVN-WebUI',
    'Version':'Forge-v2',
    'Extensions':'base',
    'Controlnet':'none',
    'Prune':True,
    'ExtraNode':False,
    'SD15model':'',
    'SDXLmodel':'',
    'Fluxmodel':'',
    'SDVNmodel':'',
    'InpaintMd':'',
    'DriveLib' : True,
    'CommandLine' : ''
}
for key, value in check_value.items():
  if key not in globals():
    globals()[key] = value

def cloudflare_thread(port):
  import socket
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()

  p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  for line in p.stderr:
    l = line.decode()
    if "trycloudflare.com " in l:
      print("Link sever sử dụng Online:", l[l.find("http"):], end='')

def localtunnel_thread(port):
  while True:
      time.sleep(0.5)
      sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
      result = sock.connect_ex(('127.0.0.1', port))
      if result == 0:
        break
      sock.close()
  print("\nKhởi chạy localtunnel Sever\n")

  print("Mật khẩu cho localtunnel là:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))
  p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
  for line in p.stdout:
    print(line.decode(), end='')

def sever_flare(port):
  threading.Thread(target=localtunnel_thread, daemon=True, args=(port,)).start()
  threading.Thread(target=cloudflare_thread, daemon=True, args=(port,)).start()

def inf(msg, style, wdth):
    inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
    display(inf)
def aria_down(link,path,name):
  !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {link} -d  {path} -o {name}

def c_folder(path):
  path = f'{User_folder}/{path}'
  os.makedirs(path, exist_ok=True)

def default_file_check(folder_source,folder_check,over=False):
  for item in os.listdir(folder_source):
    file_path = os.path.join(folder_source, item)
    if os.path.isfile(file_path):
      cp_path = file_path.replace(SDVNFolder, folder_check)
      !cp {'-f' if over else '-n'} {file_path} {cp_path}   
    elif os.path.isdir(file_path):
      default_file_check(file_path,folder_check,over)

def run_list_txt(txt):
  with open(f"{txt}", "r") as file:
    lists = file.readlines()
  for list in lists :
    !{list}

def link_folder(source,target):
  !rm -r {target}
  !ln -s {source} {target}

def split_string_at(link):
    delimiter = "@="
    if delimiter in link:
        parts = link.split(delimiter)
        if '.ckpt' not in parts[1] and '.gguf' not in parts[1] and '.safetensors' not in parts[1]:
          parts[1] = f'{parts[1]}.safetensors'
        return parts[0], parts[1]
    else:
        return link, None

def download(link):
  link = link.replace('&', '\&')
  valuelink, valuename = split_string_at(link)
  if '#' not in link:
    if valuename != None :
      aria_down(valuelink,checkpointpatch,valuename)
    else :
      !wget {"" if "?token=" in valuelink else "-c"} {valuelink} -P {checkpointpatch} --content-disposition

def download_lib(model,modellist):
  downloadcode = 'aria2c --console-log-level=error -c -x 16 -s 16 -k 1M'
  if 'https:' in model:
    download(model)
  elif model in modellist:
    if not any(ext in model for ext in ['.ckpt', '.gguf', '.safetensors']):
      model += '.safetensors'
    !{downloadcode} {modellist[model]} -d {checkpointpatch} -o {model}

def download_txt(lines):
  for line in lines:
    download(line)

def run_ver(Version,CommandLine):
  %cd {Version_folder}
  if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2":
    sever_flare(7860)
    drive_arg = f"--ckpt-dir {User_folder}/Model --embeddings-dir {User_folder}/Embeddings" if DriveLib else ""
    final_arg = f"--xformers --theme dark --enable-insecure-extension-access --disable-safe-unpickle --share --no-half-vae {CommandLine} \
                --ui-settings-file {User_folder}/Setting/WebUI_Setting.json \
                --styles-file {User_folder}/Setting/WebUI_Styles.csv --lora-dir {User_folder}/Lora {drive_arg}" 
    !python launch.py {final_arg}
  if Version == "ComfyUI":
    sever_flare(8888)
    final_arg = f"--preview-method auto --port 8888 --input-directory {User_folder}/ComfyUIinput {CommandLine}" 
    !python main.py {final_arg}
  if Version == "Kohya" :
    html="<p style='color:yellow'><code><b>Do lỗi hiển thị khi train, nên chạy lệnh sau trong cửa sổ dòng lệnh để quản lý tốt hơn :<b></code></p><h3 style='color:lightgreen'><code>[ cd KohyaUI;python kohya_gui.py --share --headless ]</code></b></h3>"
    display(HTML(html))
    !python {Version_folder}/kohya_gui.py --share --headless
  if Version == "Fooocus" :
    !python {Version_folder}/entry_with_update.py --share --theme dark --disable-preset-download {CommandLine}
  if Version == "AutoRetouch":
    !python {Version_folder}/auto_retoucher.py

def xformers_check():
  xformersver = {
    'Forge-v2':'0.0.27.post2',
    'Automatic': '0.0.23.post1',
    'Forge': '0.0.23.post1'
  }
  if Version in xformersver :
    !pip install xformers=={xformersver[Version]}
  clear_output()
  !pip show torch xformers

def install_custom():
  if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2" or Version == "ComfyUI":
    html="<p style='color:lightgreen'><b><code>Chỉnh sửa tệp Custom trong thư mục {Thư mục Data}/Setting để cài thêm extension/node tuỳ chọn tương ứng cho từng Version.</code></b></p>"
    display(HTML(html))
    print('[1;32mCài đặt Extension/Note tuỳ chọn...')
    with capture.capture_output() as cap:
      if Version != "ComfyUI" :
        %cd {Version_folder}/extensions
        exfile = "Custom_Automatic.txt" if Version == "Automatic" else "Custom_Forgev2.txt" if Version == "Forge-v2" else "Custom_Forge.txt"
        if os.path.isfile(f"{User_folder}/Setting/{exfile}"):
          with open(f"{User_folder}/Setting/{exfile}", "r") as file:
            lists = file.readlines()
        for list in lists :
          !git clone {list}
        %cd /content/SDVN
      else :
        %cd {Version_folder}/custom_nodes
        if os.path.isfile(f"{User_folder}/Setting/Custom_ComfyNode.txt"):
          with open(f"{User_folder}/Setting/Custom_ComfyNode.txt", "r") as file:
            notes = file.readlines()
          for note in notes :
            last_segment = note.rsplit('/', 1)[-1].strip()
            if not os.path.exists(f'{last_segment }'):
              !git clone {note}
              !pip install -q -r {Version_folder}/custom_nodes/{last_segment}/requirements.txt
        %cd /content/ComfyUI

    inf('\u2714 Done','success', '150px')

%cd {root_folder}
folder_list = [ '',
                'Model',
                'Lora',
                'Embeddings',
                'wildcards',
                'Hypernetworks',
                'Export',
                'Export/ComfyUI',
                'Export/Fooocus',
                'Export/AutoRetouch',
                'ControlnetModel',
                'TrainData',
                'AnimateDiffModel',
                'Lora/AnimateDiffLora',
                'ComfyUIinput',
                'Setting',
                'Setting/Comfy_Setting',
                'Setting/Comfy_Setting/default',
                'Setting/Comfy_Setting/default/workflows']
for i in folder_list:
  c_folder(i)

default_file_check(f"{SDVNFolder}/Setting",User_folder)

file_path = f'{User_folder}/Setting/WebUI_Setting.json'
with open(file_path, 'r') as file:
    json_str = file.read()
default_path = "/content/drive/MyDrive/SD-Data"
user_path = f"{User_folder}"
json_str = json_str.replace(default_path, user_path)
with open(file_path, 'w') as file:
    file.write(json_str)
clear_output()

html="<h1 style='color:lightgreen; font-size:3em' ><code>BẮT ĐẦU CÀI ĐẶT CHƯƠNG TRÌNH - VUI LÒNG ĐỢI ...</code></h1><h3><code>☕️ Pha một ly cà phê đợi 5-10 phút nữa nhé ...</code></h3><h3><code>☕️ Cài tuỳ chọn Extension/Note/ thông qua thẻ txt tương ứng trong thư mục Setting trong thư mục SD-Data</code></h3><p>_________</p>"
display(HTML(html))
image_url = 'https://github.com/StableDiffusionVN/SDVN-WebUI/blob/main/huongdan.png?raw=true'
response = requests.get(image_url)
display(Image(data=response.content))
#Start download
if Version == "Automatic" or Version == "Forge" or Version == "Forge-v2" or Version == "ComfyUI":
  if Controlnet != "none":
    if Prune:
      !git clone https://huggingface.co/StableDiffusionVN/Prunedcontrolnet {SDVNFolder}/ControlNet/Basecontrolnet
    else:
      !git clone https://huggingface.co/StableDiffusionVN/Basecontrolnet {SDVNFolder}/ControlNet/Basecontrolnet
  if Controlnet == "extra" :
    !git clone https://huggingface.co/StableDiffusionVN/Extracontrolnet {SDVNFolder}/ControlNet/Extracontrolnet
  if Controlnet == "XL" :
    !git clone https://huggingface.co/StableDiffusionVN/XLControlnet {SDVNFolder}/ControlNet/XLControlnet
  !ln -s {User_folder}/ControlnetModel {SDVNFolder}/ControlNet
  link_folder(f'{User_folder}/ControlnetModel',f'{SDVNFolder}/ControlNet/ControlnetModel')
  if Lib == True :
    !git clone https://github.com/StableDiffusionVN/WC-SDVN
    link_folder(f'{root_folder}/WC-SDVN',f'{User_folder}/wildcards/WC-SDVN')
    !git clone https://huggingface.co/StableDiffusionVN/clone
    link_folder(f'{root_folder}/clone/Negative',f'{User_folder}/Embeddings/Negative')
    link_folder(f'{root_folder}/clone/Lora',f'{User_folder}/Lora/Lora')

with capture.capture_output() as cap:
#WebUI
  if Version =="Forge" or Version =="Automatic" or Version =="Forge-v2":
    Version_folder = f"{root_folder}/SDVN"
    if test != Version:
      !rm -r /content/SDVN
    test = Version

    if Version == "Automatic":
      !git clone https://github.com/phamhungd/SDVN
    if Version == "Forge" :
      !git clone -b F-19 https://github.com/phamhungd/SDVN-Forge /content/SDVN
    if Version == "Forge-v2":
      !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge /content/SDVN

    if Version != "Forge-v2":
      Auto_ex = "SDVN-WebUI/Extension/Automatic"
      run_list_txt(f"{Auto_ex}/{'baseprund.txt' if Prune else 'exbase.txt'}")
      if Extensions !="base":
        exlist = {'plus':'explus.txt','video':'exvideo.txt','train':'extrain.txt'}
        run_list_txt(f"{Auto_ex}/{exlist[Extensions]}")
    else:
      run_list_txt(f'{SDVNFolder}/Extension/Forgev2/forge_ex.txt')

    link_folder(f'{SDVNFolder}/ControlNet',f'{Version_folder}/models/ControlNet')
    aria_down('https://huggingface.co/StableDiffusionVN/XLControlnet/resolve/main/InstantID/control_instant_id_sdxl.safetensors',f'{SDVNFolder}/ControlNet/Basecontrolnet','control_instant_id_sdxl.safetensors')
    aria_down('https://huggingface.co/StableDiffusionVN/XLControlnet/resolve/main/InstantID/adapter_instant_id_sdxl.bin',f'{SDVNFolder}/ControlNet/Basecontrolnet','adapter_instant_id_sdxl.bin')
    aria_down('https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt',f'{Version_folder}/models/VAE','VAE84.vae.pt')
    aria_down('https://huggingface.co/phamhungd/GuoZovya/resolve/main/4x-UltraSharp.ckpt',f'{Version_folder}/models/ESRGAN','4x-UltraSharp.pth')
    link_folder(f'{User_folder}/wildcards',f'{Version_folder}/extensions/Dynamic-prompts/wildcards')
    default_file_check(f"{SDVNFolder}/SDVN",root_folder,True)

    if Version=="Forge":
      !rm -r /content/SDVN/extensions/sd-webui-controlnet
      !git clone https://github.com/layerdiffusion/sd-forge-layerdiffusion {Version_folder}/extensions/sd-forge-layerdiffusion
      !pip install backend
      !git clone https://github.com/huchenlei/sd-forge-ic-light {Version_folder}/extensions/sd-forge-ic-light
      aria_down('https://huggingface.co/huchenlei/IC-Light-ldm/resolve/main/iclight_sd15_fbc_unet_ldm.safetensors',f'{Version_folder}/models/ic-light','iclight_sd15_fbc_unet_ldm.safetensors')
      aria_down('https://huggingface.co/huchenlei/IC-Light-ldm/resolve/main/iclight_sd15_fc_unet_ldm.safetensors',f'{Version_folder}/models/ic-light','iclight_sd15_fc_unet_ldm.safetensors')
#Comfy
  if Version == "ComfyUI":
    time = datetime.now().date()
    !mkdir -p {User_folder}/Export/ComfyUI/{time}

    %cd {root_folder}
    Version_folder = f"{root_folder}/ComfyUI"
    !git clone https://github.com/comfyanonymous/ComfyUI
    !pip install -q -r /content/ComfyUI/requirements.txt
    default_file_check(f"{SDVNFolder}/ComfySetting",f"{Version_folder}/web/scripts",True)

    %cd {Version_folder}/custom_nodes
    if Prune == True :
      run_list_txt(f'{SDVNFolder}/Extension/Comfy/comfynote_prune.txt')
    else :
      run_list_txt(f'{SDVNFolder}/Extension/Comfy/comfynote.txt')
    if ExtraNode == True :
      run_list_txt(f'{SDVNFolder}/Extension/Comfy/comfynote_plus.txt')

    %cd {root_folder}

    list_link_folder = {
        'Model':'models/checkpoints/Model',
        'Lora':'models/loras/Lora',
        'Embeddings':'models/embeddings/Embeddings',
        'Hypernetworks':'models/hypernetworks/Hypernetworks',
        'wildcards':'wildcards',
        'AnimateDiff':'ccustom_nodes/ComfyUI-AnimateDiff-Evolved/models/AnimateDiff',
        'Lora/AnimateDiff':'custom_nodes/ComfyUI-AnimateDiff-Evolved/motion_lora/Lora',
        'Setting/Comfy_Setting':'user',
        'Setting/Comfy_Setting/default/rgthree_config.json':'custom_nodes/rgthree-comfy/rgthree_config.json',
        'Setting/WebUI_Styles.csv':'styles/styles.csv',
        f'Export/ComfyUI/{time}':f'output',
    }

    !mkdir -p  {Version_folder}/styles
    for key, value in list_link_folder.items():
      link_folder(f'{User_folder}/{key}',f'{Version_folder}/{value}')
    link_folder(f'{SDVNFolder}/ControlNet',f'{Version_folder}/models/controlnet')
#Kohya    
  if Version == "Kohya" :
    Version_folder = f"{root_folder}/KohyaUI"
    %cd {root_folder}
    !git clone -b v24.1.4 https://github.com/phamhungd/KohyaUI
    %cd {Version_folder}
    !./setup.sh
#Fooocus
  if Version =="Fooocus" :
    Version_folder = f"{root_folder}/Fooocus"
    %cd {root_folder}
    !git clone https://github.com/lllyasviel/Fooocus
    %cd {Version_folder}
    link_folder(f'{User_folder}/Export/Fooocus',f'{Version_folder}/outputs')
    link_folder(f'{User_folder}/Lora',f'{Version_folder}/models/loras')
    !cp -f {SDVNFolder}/FooocusSetting/Fooocus_config.txt {Version_folder}/config.json
#AutoRetouch
  if Version == "AutoRetouch" :
    Version_folder = f"{root_folder}/AutoRetouch"
    !git clone https://github.com/phamhungd/auto_retoucher {Version_folder}
    !pip install -q -r {Version_folder}/requirements.txt
    link_folder(f'{User_folder}/Export/AutoRetouch',f'{Version_folder}/outputs')

inf('\u2714 Done','success', '150px')

#Custom Checkpoint download

if Version =="Forge" or Version =="Automatic" or Version == "Forge-v2":
  Version_path =f"{Version_folder}/models/Stable-diffusion/Checkpoint"
if Version == "ComfyUI" :
  Version_path = f"{Version_folder}/models/checkpoints/Checkpoint"
if Version == "Fooocus" :
  Version_path = f"{Version_folder}/models/checkpoints"
if Version == "AutoRetouch" :
  Version_path = f"{Version_folder}/models"

checkpointpatch = f"{root_folder}/Checkpoint"

file_path = f'{SDVNFolder}/model_lib.json'
with open(file_path, 'r') as json_file:
    modellist = json.load(json_file)

download_lib(SD15model,modellist)
download_lib(SDXLmodel,modellist)
download_lib(Fluxmodel,modellist)
download_lib(SDVNmodel,modellist)
download_lib(InpaintMd,modellist)

file_path = f'{User_folder}/Setting/Custom_Model_List.txt'
with open(file_path, 'r') as file:
    lines = file.readlines()
lines = [line.strip() for line in lines]
download_txt(lines)
if Version != "Kohya":
  link_folder(f'{checkpointpatch}',f'{Version_path}')
